In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np

from tensorflow.keras.models import load_model
import json
import random

import os

script_dir = os.getcwd()

model_file_path = os.path.join(script_dir, "chatbot_model.h5")
intents_file_path = os.path.join(script_dir, "intents.json")
words_file_path = os.path.join(script_dir, "words.pkl")
classes_file_path = os.path.join(script_dir, "classes.pkl")


# Load mô hình và các dữ liệu cần thiết từ các tệp đã lưu trữ trước đó
# model = load_model('/content/drive/MyDrive/code/Chatbot/chatbot_model.h5')
# intents = json.loads(open('/content/drive/MyDrive/code/Chatbot/intents.json').read())
# words = pickle.load(open('/content/drive/MyDrive/code/Chatbot/words.pkl','rb'))
# classes = pickle.load(open('/content/drive/MyDrive/code/Chatbot/classes.pkl','rb'))

model = load_model(model_file_path)
intents = json.loads(open(intents_file_path).read())
words = pickle.load(open(words_file_path, "rb"))
classes = pickle.load(open(classes_file_path, "rb"))


# Định nghĩa hàm để làm sạch câu nhập từ người dùng
lemmatizer = WordNetLemmatizer()


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# Định nghĩa hàm bow (bag of words) để chuyển đổi câu nhập thành một vector các từ
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


# Định nghĩa hàm predict_class để dự đoán nhãn (intent) của câu nhập:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


# Định nghĩa hàm getResponse để lấy câu trả lời từ dữ liệu intents dựa trên nhãn được dự đoán:
def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


# # Tạo vòng lặp vô hạn để cho phép người dùng tương tác với chatbot
# while True:
#     user_input = input("You: ")
#     if user_input.lower() == 'quit':
#         break
#     response = chatbot_response(user_input)
#     print("Bot:", response)

# Danh sách các câu hỏi được xác định trước
predefined_questions = [
    "Xin chào",
    "Có tất cả bao nhiêu giày",
    "Có tất cả bao nhiêu thương hiệu giày",
    "Có tất cả bao nhiêu khuyến mãi",
    "Tạm biệt",
]

# Duyệt qua danh sách các câu hỏi và in ra câu trả lời từ chatbot
for question in predefined_questions:
    print("You:", question)
    response = chatbot_response(question)
    print("Bot:", response)

You: Xin chào
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Bot: Xin chào, Shop có thể giúp gì cho bạn?
You: Các đôi giày có trong shop
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Bot: Adidas Superstar - Adidas, Size: 42, Color: Blue, Discounted Price: 1800000.0 VND, Promotion: Quay Trở Lại Trường Học, Start Date: 01-09-2024, End Date: 30-09-2024, Discount: 10%
You: Cho tôi một đôi Nike
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Bot: Puma Suede Classic+ - Puma, Size: 43, Color: Yellow, Discounted Price: 1800000.0 VND, Promotion: Quay Trở Lại Trường Học, Start Date: 01-09-2024, End Date: 30-09-2024, Discount: 10%
You: Tạm biệt
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Bot: Xin chào, Shop có thể giúp gì cho bạn?
